<a href="https://colab.research.google.com/github/boyonfix/ryanellegood/blob/main/SubText_Working.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing Google Python Client, Google OAUTH2 Verification, OpenAI

In [ ]:
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib
!pip install openai

# Importing Relevant Libraries & Authenticating Google Sheets and OpenAi Keys

In [ ]:
from google.colab import auth
import google.auth
from googleapiclient.discovery import build
from openai import OpenAI
from google.colab import userdata
from IPython.display import Image, display, Markdown
import time
import textwrap

# Authenticate with Google Sheets and Drive
auth.authenticate_user()
creds, _ = google.auth.default()
sheets_service = build('sheets', 'v4', credentials=creds)

# Initialize OpenAI API key
openai.api_key = userdata.get('Bok_Key')



# Image Generation, Image Description, and Storage in Google Sheet

In [ ]:
# Wrapping text function
def print_wrapped(text, width=80):
    wrapped_text = textwrap.fill(text, width=width)
    print(wrapped_text)

# Image generation function
def image_from_prompt(prompt):
    client = OpenAI(api_key=OPENAI_API_KEY)
    imageresponse = client.images.generate(
        model="dall-e-3",
        quality="hd",
        prompt=prompt,
        n=1,
        size="1024x1024"
    )

    image_url = imageresponse.data[0].url
    revised_prompt = imageresponse.data[0].revised_prompt

    # Display image and prompt in Colab
    display(Image(url=image_url, width=500))
    display(Markdown("## Prompt"))
    print_wrapped(prompt)
    display(Markdown("## Revised Prompt"))
    print_wrapped(revised_prompt)

    # Save image data to Google Sheets
    spreadsheet_id = '1T9EmLzhNYXdaaoWnjqXv6Fh2DaQCm2T3qVVUrDBVZdY'  # Replace with your actual Google Sheets ID
    sheet_name = 'Sheet1'
    range_name = f'{sheet_name}!A2:D'

    values = [
        [prompt[:20].replace(" ", "-") + "-" + str(int(time.time())), prompt, revised_prompt, image_url]
    ]
    body = {
        'values': values
    }
    result = sheets_service.spreadsheets().values().append(
        spreadsheetId=spreadsheet_id, range=range_name,
        valueInputOption='USER_ENTERED', body=body).execute()
    print(f"{result.get('updates').get('updatedRows')} rows appended.")

    return {
        "InitialURL": image_url,
        "Name": values[0][0],
        "Prompt": values[0][1],
        "RevisedPrompt": revised_prompt
    }

# Description generation from image
def description_from_image(image_data, vision_prompt):
    # Ensure image_data is valid
    if not image_data or "InitialURL" not in image_data:
        print("Error: image_data is invalid.")
        return

    client = OpenAI(api_key=OPENAI_API_KEY)

    # OpenAI call for description generation
    response = client.ChatCompletion.create(
        model="gpt-4-vision-preview",
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": vision_prompt},
                    {
                        "type": "image_url",
                        "image_url": {"url": image_data["InitialURL"]},
                    },
                ],
            }
        ],
        max_tokens=300
    )

    vision_description = response.choices[0].message.content

    display(Markdown("## Vision Description"))
    print_wrapped(vision_description)

    # Append the vision description to Google Sheets
    range_name = f"{sheet_name}!E2:E"
    result = sheets_service.spreadsheets().values().append(
        spreadsheetId=spreadsheet_id, range=range_name,
        valueInputOption='USER_ENTERED', body={'values': [[vision_description]]}).execute()
    print("Record updated successfully.")

    return vision_description

In [39]:
prompt = "A colorful and whimsical landscape with floating islands and magical creatures"
image_data = image_from_prompt(prompt)

## Prompt

A colorful and whimsical landscape with floating islands and magical creatures


## Revised Prompt

Imagine an enchanting scene with a myriad of bright, lively colors. The main
focus of the image is a fantastical landscape made up of a series of floating
islands, each unique in shape and size. These islands have vibrant and peculiar
vegetation, brimming with life. They float effortlessly in the azure sky amidst
fluffy white clouds. Surrounding the islands are diverse magical creatures, a
mélange of the familiar and the fantastical — unicorns with iridescent manes,
gryphons majestically soaring, diminutive fairies fluttering around with
delicate wings sparkling in the sunlight. The overall aesthetic suggests a
whimsical and magical universe.
1 rows appended.


Recursive Time